In [1]:
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import copy
import numpy as np
import pickle
plt.rcParams.update({'font.size': 12})
import pandas as pd
from scipy.optimize import minimize
import csv

In [2]:
def find_closest_node_to_center(node_coords):
    # create array with IDs and coordinates
    num_nodes = node_coords.shape[0]
    node_ids = np.arange(num_nodes)
    geometry = np.column_stack((node_ids, node_coords))

    # find center of geometry
    center = np.mean(geometry[:, 1:], axis=0)
    
    # find closest node to center
    distances = np.linalg.norm(geometry[:, 1:] - center, axis=1)
    closest_node_idx = np.argmin(distances)
    closest_node_id = geometry[closest_node_idx, 0]

    return int(closest_node_id)

def poly_vert(myRVE): # Find vertices that make up the polyhedra
    n_poly = myRVE.n_poly
    poly_face_f_all = myRVE.poly_face_f_all
    face_vert_f_all = myRVE.face_vert_f_all
    verts_poly = []

    for i in range(n_poly): #Loop over polyhedrons
        #verts = [] # Vertices IDs
        verts = np.array([])
        current_poly_faces = poly_face_f_all[i]
        # let's give a max here and break if possible or do nothing 
        for j in range(len(current_poly_faces)): #Loop over faces
            current_face_vert = face_vert_f_all[int(current_poly_faces[j])-1]
            #current_face_edge = face_edge_f_all[int(current_poly_faces[j])-1]
            for m in range(len(current_face_vert)):
                verts = np.hstack((verts,current_face_vert[m]))
                
        verts_poly.append(np.unique(verts))     
           
    return verts_poly

def Vol_SA_Dist(node_pos,myRVE):
    #node_pos = jnp.array(node_pos)
    n_poly = myRVE.n_poly
    #poly_face_f_all = myRVE.poly_face_f_all
    #face_vert_f_all = myRVE.face_vert_f_all
    edge_pos = myRVE.edge_pos
    v_poly = myRVE.v_poly
    
    poly_vol = np.zeros((n_poly,1))
    poly_SA = np.zeros((n_poly,1))

    kappa = 1. # Strength of convexity constraint 
    convex = 0. # We want to keep this value equal to zero to be sure all the cells and the geometry is convex
           
    for i in range(n_poly):
        #print(v_poly)
        current_v_poly = v_poly[i]
        #print(current_v_poly)
        v_all = node_pos[current_v_poly.astype(int)-1]
        hull = ConvexHull(v_all)
        poly_vol[i] = hull.volume
        poly_SA[i] = hull.area
        if hull.vertices.size != len(v_all):
            convex += kappa  
            
    v1_coords = node_pos[edge_pos[:,0].astype(int)-1] # Coordinates of the first points
    v2_coords = node_pos[edge_pos[:,1].astype(int)-1] # Coordinates of the second points
    dist = ((v2_coords[:,0]-v1_coords[:,0])**2 + (v2_coords[:,1]-v1_coords[:,1])**2 + (v2_coords[:,2]-v1_coords[:,2])**2)**0.5
    edge_distance = dist.astype(float).reshape(-1,1)

    return poly_vol, poly_SA, edge_distance, convex

## Functions to calculate the energy
def evalPsi(node_x,myRVE):
    
    kvol = myRVE.k_vol
    karea = myRVE.k_area
    klength = myRVE.k_length
    ncell = myRVE.n_poly
    
    # compute the deformed volumes, areas and lengths 
    # Evaluate volume, surface area and edge length for each cell
    vol_def,SA_def,length_def,convex_def = Vol_SA_Dist(node_x,myRVE)
    
    # energy is going to be three components 
    Jn = vol_def/vol_ref
    Ja = SA_def/SA_ref
    Psi_vol = kvol*np.mean((Jn-1)**2)
    Psi_area = karea*np.mean((Ja-1)**2)    
    Psi_length = 0    
    
    return Psi_vol,Psi_area,Psi_length#+convex_def

def Dist(node_pos,myRVE):

    edge_pos = myRVE.edge_pos
        
    v1_coords = node_pos[edge_pos[:,0].astype(int)-1] # Coordinates of the first points
    v2_coords = node_pos[edge_pos[:,1].astype(int)-1] # Coordinates of the second points
    dist = ((v2_coords[:,0]-v1_coords[:,0])**2 + (v2_coords[:,1]-v1_coords[:,1])**2 + (v2_coords[:,2]-v1_coords[:,2])**2)**0.5
    edge_distance = dist.astype(float).reshape(-1,1)

    return edge_distance

## Functions to calculate the derivatives of the energy
def evaldPsidx(node_pos,myRVE):
    #node_pos = jnp.array(node_pos)
    n_poly = myRVE.n_poly
    #poly_face_f_all = myRVE.poly_face_f_all
    #face_vert_f_all = myRVE.face_vert_f_all
    edge_pos = myRVE.edge_pos
    v_poly = myRVE.v_poly
    
    poly_vol = np.zeros((n_poly,1))
    poly_SA = np.zeros((n_poly,1))

    kappa = 1. # Strength of convexity constraint 
    convex = 0. # We want to keep this value equal to zero to be sure all the cells and the geometry is convex
       
    ##---------------------------------------##
    # initialize the residual vector, three, 
    # let's do one per metric of interest
    n_node = len(node_pos)
    # note I am going to flatten it, every node has 3 coords
    # so the number of unknowns is 3*n_node 
    # the residual, just like in FE will have 3*n_node entries 
    #dvoldx = np.zeros((n_node*3)) 
    #dareadx = np.zeros((n_node*3))
    #dedgedx = np.zeros((n_node*3))
    dPsidx_vol = np.zeros((n_node,3)) 
    dPsidx_area = np.zeros((n_node,3)) 
    dPsidx_length = np.zeros((n_node,3))
    ##---------------------------------------## 
    
    kvol = myRVE.k_vol
    karea = myRVE.k_area
    klength = myRVE.k_length
    #print(kvol,karea,klength)
    
    for i in range(n_poly):
        current_v_poly = v_poly[i]
        #print(current_v_poly)
        v_all = node_pos[current_v_poly.astype(int)-1]
        hull = ConvexHull(v_all)
        poly_vol[i] = hull.volume
        poly_SA[i] = hull.area
        if hull.vertices.size != len(v_all):
            convex += kappa  
        ###### PROF. CODE
        eps = 1e-6
        for ni in range(len(v_all)):
            global_node_ni = current_v_poly[ni].astype(int)-1
            # loop over coordinate
            for ci in range(3):
                # some variations for central difference
                v_all[ni,ci] += eps
                # this is the change in volume by moving the node by a little bit
                # namely f(x+eps,y) where y represents all other nodal values which stay constant
                hull_p_nici = ConvexHull(v_all)
                # for central difference 
                v_all[ni,ci] -= eps
                # apply finite difference scheme
                # use the change in volume to get derivative 
                dpsivoldnici = 2*kvol*(poly_vol[i]-vol_ref[i])*(hull_p_nici.volume - poly_vol[i])/(eps)/len(vol_ref)
                # this is the change in area
                dpsiareadnici = 2*karea*(poly_SA[i]-SA_ref[i])*(hull_p_nici.area  - poly_SA[i])/(eps)/len(SA_ref)
                # now, the thing is how to assing this to the GLOBAL node 
                # add to residual since the derivative from multiple polyhedra can add to the
                # same nodal entry
                # JACQUES: you need to know what is the global index of the node
                # see how we are moving node 'ni' of v_all, but what node is that in the
                # node_pos array? 
                dPsidx_vol[global_node_ni,ci] += dpsivoldnici
                dPsidx_area[global_node_ni,ci] += dpsiareadnici 
        # MY CODE. I can get away with this since a shift on the hull does not change the volume or area
        #eps = 1e-6
        #global_node_ni = current_v_poly.astype(int)-1
        #dPsidx_vol[global_node_ni,:] += 2*kvol*(poly_vol[i]-vol_ref[i])*(poly_vol[i] - poly_vol[i])/(eps)
        #dPsidx_area[global_node_ni,:] += 2*karea*(poly_SA[i]-SA_ref[i])*(poly_SA[i]  - poly_SA[i])/(eps)
        
             
    #edge_distance = Dist(node_pos,myRVE)
                              
    #dPsi_dlength = 2.*klength*(edge_distance - length_ref)*(node_pos[edge_pos[:,1].astype(int)-1] - node_pos[edge_pos[:,0].astype(int)-1])/(edge_distance)/len(edge_distance) 
    #for ni in range(len(edge_distance)):
    #    n1 = edge_pos[ni,0].astype(int)-1
    #    n2 = edge_pos[ni,1].astype(int)-1
    #    dPsidx_length[n1,:] -= dPsi_dlength[ni,:]
    #    dPsidx_length[n2,:] += dPsi_dlength[ni,:]
    
    Psidx_length = 0.
    return dPsidx_vol, dPsidx_area, dPsidx_length

def calcBCforce(node_def,myRVE,stretch,beta):
    ## force vector for boundary conditions 
    BC_forces = np.zeros((len(node_def),3))
    # over-write the position of the nodes that we can apply BC to 
    PBC_penalty = 0
    #beta = 10000 # penalty to try to enforce the constraint 
    #dPBC = [0.01,0.01,0.01] # the displacements we want to apply 
    l0 = 0.5 # RVE length
    dPBC = np.array([1,1,1])*(l0*stretch-l0) # the displacements we want to apply 
    BC_pair = myRVE.vert_pair
    n_BC = len(BC_pair)
    for i in range(n_BC):
        n2 = int(BC_pair[i,0])-1
        n1 = int(BC_pair[i,1])-1
        #print(node_x[n2,1]-node_x[n1,1])
        for j in range(3):
            # I dont think this is correct
            #PBC_penalty += beta*((node_x[n2,j]-node_x[n1,j])-(dPBC[j] + l0)*BC_pair[i,j+2])**2
            BC_forces[n1,j] += -2*beta*((node_def[n2,j]-node_def[n1,j])-(dPBC[j] + l0)*BC_pair[i,j+2])
            BC_forces[n2,j] += +2*beta*((node_def[n2,j]-node_def[n1,j])-(dPBC[j] + l0)*BC_pair[i,j+2])
    #print(BC_forces)
            
    #n2 = BC_pair[:,0].astype(int)-1
    #n1 = BC_pair[:,1].astype(int)-1

    # calculate the BC forces using arrays instead of a loop
    #BC_forces[n1,:] -= 2*beta*((node_def[n2,:]-node_def[n1,:])-(dPBC + l0)*BC_pair[:,2:5])
    #BC_forces[n2,:] += 2*beta*((node_def[n2,:]-node_def[n1,:])-(dPBC + l0)*BC_pair[:,2:5])
            
    return BC_forces


In [3]:
# Load the CSV file using loadtxt
data = np.loadtxt('/scratch/bell/jbarsima/adip_size/tess_files/samples.csv', delimiter=',')
#data = data.transpose()
Din = data[:,0]
Dstdin = data[:,1]
Sin = data[:,2]
Sstdin = data[:,3]

In [4]:
D = np.hstack((Din,Din))
D = np.hstack((D,Din))

Dstd = np.hstack((Dstdin,Dstdin))
Dstd = np.hstack((Dstd,Dstdin))

S = np.hstack((Sin,Sin))
S = np.hstack((S,Sin))

Sstd = np.hstack((Sstdin,Sstdin))
Sstd = np.hstack((Sstd,Sstdin))

In [ ]:
nodes_all_fin = []
Psi_all = []
Psi_vol_all = []
Psi_area_all = []
Psi_length_all = []
Dc = []
Dstdc = []
Sc = []
Sstdc = []
vol_all = []
ncell_all = []
node_x = []
iii = 0

for RVE_N in range(0,1200):
    NN = str(RVE_N+1)
    print('RVE',NN)
    file = '/scratch/bell/jbarsima/adip_size/tess_files/RVE_'+ NN +'.tess'
    #file = 'RVE_2.tess'
    data = []
    with open(file, 'r') as f:
        for line in f:
            line_data = line.strip().split()
            # append the data from this line to the list
            data.append(line_data)

    ## VERTEX DATA
    # loop through each element of the 'data' list
    for i in range(len(data)):
        # check if the current element matches the search value
        if data[i] == ['**vertex']:
            #print(f"Found '**vertex' at index {i}.")
            vertex_in = i #Index for vertex
            break

    n_vertex = int(data[vertex_in + 1][0]) #Number of vertices

    node_ref = np.zeros((1,3))
    for i in range(n_vertex):
        ii = i + vertex_in + 2
        x = float(data[ii][1])
        y = float(data[ii][2])
        z = float(data[ii][3])
        temp = np.array([x,y,z])
        node_ref = np.vstack((node_ref,temp))
    node_ref = node_ref[1:,:] #Vertices

    ## EDGE DATA
    # loop through each element of the 'data' list
    for i in range(n_vertex,len(data)):
        # check if the current element matches the search value
        if data[i] == ['**edge']:
            #print(f"Found '**edge' at index {i}.")
            edge_in = i #Index for edge

    n_edge = int(data[edge_in + 1][0]) #Number of edges

    edge_pos = np.zeros((1,2))
    for i in range(n_edge):
        ii = i + edge_in + 2
        ini_ = int(data[ii][1])
        end_ = int(data[ii][2])
        temp = np.array([ini_,end_])
        edge_pos = np.vstack((edge_pos,temp))
    edge_pos = edge_pos[1:,:] #edges

    ## FACE DATA
    # loop through each element of the 'data' list
    for i in range(n_edge + edge_in,len(data)):
        # check if the current element matches the search value
        if data[i] == ['**face']:
            #print(f"Found '**face' at index {i}.")
            face_in = i #Index for face
            break

    n_face = int(data[face_in + 1][0]) #Number of faces

    off = 2
    face_vert_f_all = []
    face_edge_f_all = []
    for i in range(n_face):
        ii = 4*i + face_in + off
        face_vert = data[ii] #Vertices that form the face
        face_edge = data[ii+1] #Edges that form the face
        face_vert_f = [] #Float
        face_edge_f = [] #Float
        for j in range(len(face_vert)):
            face_vert_f.append(float(face_vert[j]))
        face_vert_f = np.abs(face_vert_f[2:])
        # Array that containts all the vertices for the faces. Useful to find volume without
        # having to index edges
        face_vert_f_all.append(face_vert_f) 
        for m in range(len(face_edge)):
            face_edge_f.append(float(face_edge[m]))
        face_edge_f = np.abs(face_edge_f[1:])
        # Array that containts all the edges for the faces
        # Note that this has "the same" data as face_vert_f_all but 
        # with edge information instead of vertex information directly.
        # This one can be used to create a mesh as it lists the node
        # connectivity by edge. They are ordered correctly so they can be indexed.
        # For example, face 4 contains edges [4. 25. 26. 15. 27. 28.] which have nodes
        # [8. 2. 4. 1. 14. 15.], in that order. These edges can be used to index
        # the array edge_pos. For example, edge_pos[3] will give the nodes that make up
        # edge 4 (8 2). Then, using this data, one can index node_ref to obtain the coordinates
        # of the vertex. THIS PARTICULAR NUMBERS WERE OBTAINED FROM RVE_2
        face_edge_f_all.append(face_edge_f) 
    #print(face_edge_f_all[3])
    #print(face_vert_f_all[3])
    #print(edge_pos[3])
    #print(node_ref[7])

    ## POLYHEDRON DATA
    # loop through each element of the 'data' list
    for i in range(n_face + face_in,len(data)):
        # check if the current element matches the search value
        if data[i] == ['**polyhedron']:
            #print(f"Found '**polyhedron' at index {i}.")
            poly_in = i #Index for face
            break

    n_poly = int(data[poly_in + 1][0]) #Number of polyhedrons
    
    if n_poly >= 4000:
        print('Too many cells')
        continue

    poly_face_f_all = []
    for i in range(n_poly):
        ii = i + poly_in + 2
        poly_face = data[ii] #Faces that form the poly
        poly_face_f = [] #Float
        for j in range(len(poly_face)):
            poly_face_f.append(float(poly_face[j]))
        poly_face_f = np.abs(poly_face_f[2:]) 
        poly_face_f_all.append(poly_face_f) #Array that containts all the faces
    #print(poly_face_f_all)

    ##############################################################
    ## PERIODICITY INFO
    ## PERIODICITY SECTION INDEX
    # loop through each element of the 'data' list
    for i in range(n_poly + poly_in,len(data)):
        # check if the current element matches the search value
        if data[i] == ['**periodicity']:
            #print(f"Found '**periodicity' at index {i}.")
            periodicity_in = i #Index for periodicity section
            break

    ## VERTEX INFO
    # loop through each element of the 'data' list
    for i in range(periodicity_in,len(data)):
        # check if the current element matches the search value
        if data[i] == ['*vertex']:
            #print(f"Found '*vertex' at index {i}.")
            vertex_per_in = i #Index for periodicity section
            break
    n_vertex_per = int(data[vertex_per_in + 1][0]) #Number of periodic vertices

    vert_pair = np.zeros((n_vertex_per,5)) # <secondary_ver_id> <primary_ver_id> <per_shift_x> <per_shift_y> <per_shift_z>
    vert_pair_coord = np.zeros((1,3)) # Coordinates of the periodic vertices. Twice the length of vert_pair always
    # Ordered as <secondary_ver_coords> in the 2n row, <primary_ver_coords> in the 2n+1 row. n = 0,1,2,...
    for i in range(n_vertex_per):
        ii = i + vertex_per_in + 2
        vert_pair_temp = data[ii]  
        vert_pair[i,0] = vert_pair_temp[0] #Secondary vertex
        vert_pair[i,1] = vert_pair_temp[1] #Primary vertex
        vert_pair[i,2] = vert_pair_temp[2] #Periodic shift x
        vert_pair[i,3] = vert_pair_temp[3] #Periodic shift y
        vert_pair[i,4] = vert_pair_temp[4] #Periodic shift z
        #print(vert_pair_temp)
        vert_pair_coord = np.vstack((vert_pair_coord,node_ref[int(vert_pair[i,0])-1]))
        vert_pair_coord = np.vstack((vert_pair_coord,node_ref[int(vert_pair[i,1])-1]))
    vert_pair_coord = vert_pair_coord[1:,:]
    #print(len(vert_pair_coord))

    ## EDGE INFO
    # loop through each element of the 'data' list
    for i in range(n_vertex_per + vertex_per_in,len(data)):
        # check if the current element matches the search value
        if data[i] == ['*edge']:
            #print(f"Found '*edge' at index {i}.")
            edge_per_in = i #Index for periodicity section
            break
    n_edge_per = int(data[edge_per_in + 1][0]) #Number of periodict vertices

    edge_pair = np.zeros((n_edge_per,2)) # Pair of periodic edges
    for i in range(n_edge_per):
        ii = i + edge_per_in + 2
        edge_pair_temp = data[ii] 
        edge_pair[i,0] = edge_pair_temp[0]
        edge_pair[i,1] = edge_pair_temp[1]
        #print(edge_pair_temp)
    #print(edge_pair)   

    ## FACE INFO
    # loop through each element of the 'data' list
    for i in range(n_edge_per + edge_per_in,len(data)):
        # check if the current element matches the search value
        if data[i] == ['*face']:
            #print(f"Found '*face' at index {i}.")
            face_per_in = i #Index for periodicity section
            break
    n_face_per = int(data[face_per_in + 1][0]) #Number of periodic faces

    face_pair = np.zeros((n_face_per,2)) # Pair of periodic faces
    for i in range(n_face_per):
        ii = i + face_per_in + 2
        face_pair_temp = data[ii]
        face_pair[i,0] = face_pair_temp[0]
        face_pair[i,1] = face_pair_temp[1]
        #print(face_pair_temp)
    #print(face_pair)  

    #Find the closest node to the center of the geometry
    ID_center = find_closest_node_to_center(node_ref)

    # initial guess for deformed RVE is just the original RVE 
    node_def = copy.deepcopy((node_ref))

    ## trying to create a hashable object 
    class RVE:
        def __init__(self,n_poly,poly_face_f_all,face_vert_f_all,face_edge_f_all,edge_pos,node_ref,vert_pair,ID_center,n_edge):
            self.n_poly = n_poly
            self.poly_face_f_all = poly_face_f_all
            self.face_vert_f_all = face_vert_f_all
            self.face_edge_f_all = face_edge_f_all
            self.edge_pos = edge_pos
            self.node_ref = node_ref
            self.vert_pair = vert_pair
            self.ID_center = ID_center
            self.n_edge = n_edge
            self.v_poly = 0. #Dummy variable for until we find the actual value
            self.k_vol = 1.
            self.k_area = 1.
            self.k_length = 1.

    myRVE = RVE(n_poly,poly_face_f_all,face_vert_f_all,face_edge_f_all,edge_pos,node_ref,vert_pair,ID_center,n_edge)
    hash(myRVE)

    poly_v = poly_vert(myRVE)
    myRVE.v_poly = poly_v # Replace dummy variable for actual value

    # Evaluate volume, surface area and edge length for each cell
    # We need to run this line in order to obtain the values for the reference configuration
    # Here I am using the Vol_SA_Dist on purpose to find the reference volume, SA, etc. quickly
    # as it is needed later.
    vol_ref,SA_ref,length_ref,convex_ref = Vol_SA_Dist(node_ref,myRVE)

    node_def = copy.deepcopy((node_ref))

    #myRVE.k_vol = 0.01531663075071653
    #myRVE.k_area = 2.009900014873474
    #myRVE.k_length = 1.0001975079890093
    myRVE.k_vol = 0.015
    myRVE.k_area = 0.035
    myRVE.k_length = 0.493591321443743
    #myRVE.k_vol = 1.0
    #myRVE.k_area = 2.0
    #myRVE.k_length = 0.09
    dt = 0.0015
    tol =  1e-5
    err = 10
    err_old = 20
    it = 0
    conv_it = 0
    
    if len(vol_ref) >= 2500:
        itmax = int(200)
    elif len(vol_ref) >= 1000:
        itmax = int(275)
    else:
        itmax = int(600)
        
    print('itmax:', itmax, 'Cells:', n_poly)
    #itmax = 1
    while err > tol and it < itmax:
        dPsisV,dPsisA,dPsisL = evaldPsidx(node_def,myRVE)
        BC_forces = calcBCforce(node_def,myRVE,1.05-10**-6,100.)
        step = -dPsisV -dPsisA -dPsisL - BC_forces
        err = np.linalg.norm(dt*(step))
        if err > err_old and it > 0:
            dt -= 0.0005
            print('Decreased time step')
            if dt <= 0:
                print('Did not converge!',dt)
                conv_it = 1
                break
        else:
            node_def += dt*(step)
            err_old = copy.deepcopy(err)
        if ((it+1) % 275) == 0 or it == 0 or (it+1) == itmax: 
            print('it:', it+1, 'dt:', dt,'error:', err)
        it += 1
    if conv_it == 0:
        vol_fin,SA_fin,length_fin,convex_fin = Vol_SA_Dist(node_def,myRVE)
        #print(convex_fin)
        Psi_fin_vol,Psi_fin_area,Psi_fin_length = evalPsi(node_def,myRVE)
        Psi_vol_all.append(Psi_fin_vol)
        Psi_area_all.append(Psi_fin_area)
        Psi_length_all.append(Psi_fin_length)
        Psi_all.append(Psi_fin_vol+Psi_fin_area+Psi_fin_length)
        nodes_all_fin.append(node_def)
        vol_all.append(np.sum(vol_fin))
        Dc.append(D[RVE_N])
        Dstdc.append(Dstd[RVE_N])
        Sc.append(S[RVE_N])
        Sstdc.append(Sstd[RVE_N])
        ncell_all.append(n_poly)
        node_x.append(node_ref)

        with open('RVE_Psi_minus_1p05.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            #writer.writerow(['D', 'Dstd', 'S', 'Sstd','Psi'])
            #for i in range(len(Dc)):
            writer.writerow([Dc[iii], Dstdc[iii], Sc[iii], Sstdc[iii],Psi_all[iii],Psi_vol_all[iii],Psi_area_all[iii],Psi_length_all[iii],vol_all[iii],ncell_all[iii]])  
        
        iii += 1

RVE 1
itmax: 600 Cells: 659
it: 1 dt: 0.0015 error: 0.4658366876779946
RVE 2
itmax: 275 Cells: 2494
it: 1 dt: 0.0015 error: 0.6846668888499869
RVE 3
itmax: 600 Cells: 291
it: 1 dt: 0.0015 error: 0.3475919548095594
RVE 4
Too many cells
RVE 5
Too many cells
RVE 6
Too many cells
RVE 7
itmax: 275 Cells: 1164
it: 1 dt: 0.0015 error: 0.5483616248610825
RVE 8
Too many cells
RVE 9
itmax: 275 Cells: 1963
it: 1 dt: 0.0015 error: 0.6461198036486727
RVE 10
itmax: 600 Cells: 198
it: 1 dt: 0.0015 error: 0.34483002123882106
RVE 11
itmax: 600 Cells: 162
it: 1 dt: 0.0015 error: 0.2898864235068892
RVE 12
itmax: 600 Cells: 258
it: 1 dt: 0.0015 error: 0.39756279700598957
RVE 13
itmax: 600 Cells: 128
it: 1 dt: 0.0015 error: 0.27103424420059075
RVE 14
itmax: 200 Cells: 3634
it: 1 dt: 0.0015 error: 0.7893747999203242
RVE 15
itmax: 275 Cells: 1001
it: 1 dt: 0.0015 error: 0.5147105036872434
RVE 16
itmax: 200 Cells: 3311
it: 1 dt: 0.0015 error: 0.758412481259426
RVE 17
itmax: 600 Cells: 108
it: 1 dt: 0.0015 err

RVE 125
Too many cells
RVE 126
itmax: 200 Cells: 2573
it: 1 dt: 0.0015 error: 0.6945363231631406
RVE 127
itmax: 200 Cells: 3060
it: 1 dt: 0.0015 error: 0.8056668264609336
RVE 128
itmax: 275 Cells: 1026
it: 1 dt: 0.0015 error: 0.5561026250825695
RVE 129
itmax: 600 Cells: 200
it: 1 dt: 0.0015 error: 0.35241315533004836
RVE 130
itmax: 600 Cells: 733
it: 1 dt: 0.0015 error: 0.45161304407820807
RVE 131
itmax: 600 Cells: 125
it: 1 dt: 0.0015 error: 0.2657953611864802
RVE 132
itmax: 600 Cells: 294
it: 1 dt: 0.0015 error: 0.36863879889989976
RVE 133
itmax: 600 Cells: 427
it: 1 dt: 0.0015 error: 0.40899976597415755
RVE 134
itmax: 600 Cells: 933
it: 1 dt: 0.0015 error: 0.5036639282313908
RVE 135
itmax: 600 Cells: 525
it: 1 dt: 0.0015 error: 0.4827946520967646
RVE 136
itmax: 275 Cells: 1196
it: 1 dt: 0.0015 error: 0.5575168756408517
RVE 137
itmax: 600 Cells: 167
it: 1 dt: 0.0015 error: 0.3210075981051238
RVE 138
itmax: 600 Cells: 243
it: 1 dt: 0.0015 error: 0.34089272733000003
RVE 139
itmax: 200 

RVE 245
itmax: 600 Cells: 694
it: 1 dt: 0.0015 error: 0.4713582635203031
RVE 246
itmax: 600 Cells: 161
it: 1 dt: 0.0015 error: 0.28081906190918465
RVE 247
itmax: 200 Cells: 2811
it: 1 dt: 0.0015 error: 0.735750192807675
RVE 248
itmax: 600 Cells: 125
it: 1 dt: 0.0015 error: 0.2798157751652335
RVE 249
itmax: 600 Cells: 301
it: 1 dt: 0.0015 error: 0.334059180452567
RVE 250
itmax: 600 Cells: 141
it: 1 dt: 0.0015 error: 0.2613135444773489
RVE 251
itmax: 600 Cells: 537
it: 1 dt: 0.0015 error: 0.45779811771549367
RVE 252
itmax: 600 Cells: 871
it: 1 dt: 0.0015 error: 0.5104307769954999
RVE 253
itmax: 600 Cells: 542
it: 1 dt: 0.0015 error: 0.4360245349473474
RVE 254
itmax: 600 Cells: 312
it: 1 dt: 0.0015 error: 0.4062399519569877
RVE 255
itmax: 600 Cells: 170
it: 1 dt: 0.0015 error: 0.2859794197712757
RVE 256
itmax: 600 Cells: 242
it: 1 dt: 0.0015 error: 0.31801486528219974
RVE 257
itmax: 600 Cells: 577
it: 1 dt: 0.0015 error: 0.42584356995561246
RVE 258
itmax: 600 Cells: 289
it: 1 dt: 0.0015 e

RVE 366
Too many cells
RVE 367
itmax: 275 Cells: 1124
it: 1 dt: 0.0015 error: 0.5567091723907428
RVE 368
itmax: 600 Cells: 162
it: 1 dt: 0.0015 error: 0.2834109860822804
RVE 369
itmax: 600 Cells: 564
it: 1 dt: 0.0015 error: 0.4473583514142277
RVE 370
itmax: 600 Cells: 216
it: 1 dt: 0.0015 error: 0.3504926333735522
RVE 371
itmax: 275 Cells: 1946
it: 1 dt: 0.0015 error: 0.6412262102132751
RVE 372
itmax: 600 Cells: 132
it: 1 dt: 0.0015 error: 0.2630299080478785
RVE 373
itmax: 600 Cells: 232
it: 1 dt: 0.0015 error: 0.32153283586805825
RVE 374
itmax: 600 Cells: 527
it: 1 dt: 0.0015 error: 0.44089933553789856
RVE 375
itmax: 600 Cells: 106
it: 1 dt: 0.0015 error: 0.2986786425837736
RVE 376
itmax: 600 Cells: 88
it: 1 dt: 0.0015 error: 0.22248150489377824
RVE 377
Too many cells
RVE 378
itmax: 600 Cells: 812
it: 1 dt: 0.0015 error: 0.4687256250599784
RVE 379
itmax: 600 Cells: 92
it: 1 dt: 0.0015 error: 0.2706188658044658
RVE 380
itmax: 600 Cells: 814
it: 1 dt: 0.0015 error: 0.5292577775640545
RV

RVE 488
itmax: 600 Cells: 781
it: 1 dt: 0.0015 error: 0.48673968719919536
RVE 489
itmax: 600 Cells: 269
it: 1 dt: 0.0015 error: 0.3812403258551497
RVE 490
itmax: 275 Cells: 1628
it: 1 dt: 0.0015 error: 0.6406118826279722
RVE 491
itmax: 275 Cells: 1161
it: 1 dt: 0.0015 error: 0.5318023007783355
RVE 492
itmax: 200 Cells: 3806
it: 1 dt: 0.0015 error: 0.7978092123149505
RVE 493
Too many cells
RVE 494
itmax: 600 Cells: 86
it: 1 dt: 0.0015 error: 0.2558757140218477
RVE 495
Too many cells
RVE 496
itmax: 275 Cells: 2422
it: 1 dt: 0.0015 error: 0.6747364586874822
RVE 497
itmax: 600 Cells: 734
it: 1 dt: 0.0015 error: 0.4467292435914964
RVE 498
itmax: 600 Cells: 100
it: 1 dt: 0.0015 error: 0.2716561209130405
RVE 499
itmax: 200 Cells: 3434
it: 1 dt: 0.0015 error: 0.7716467337701918
RVE 500
itmax: 600 Cells: 195
it: 1 dt: 0.0015 error: 0.3079508183751678
RVE 501
itmax: 200 Cells: 3987
it: 1 dt: 0.0015 error: 0.8551144661929702
RVE 502
itmax: 600 Cells: 249
it: 1 dt: 0.0015 error: 0.3554327595348180

it: 1 dt: 0.0015 error: 0.856626025050332
RVE 608
itmax: 600 Cells: 547
it: 1 dt: 0.0015 error: 0.4051307623047843
RVE 609
itmax: 600 Cells: 279
it: 1 dt: 0.0015 error: 0.3325403382211483
RVE 610
itmax: 600 Cells: 159
it: 1 dt: 0.0015 error: 0.2967894481534604
RVE 611
Too many cells
RVE 612
Too many cells
RVE 613
itmax: 600 Cells: 189
it: 1 dt: 0.0015 error: 0.34856152978864546
RVE 614
itmax: 600 Cells: 497
it: 1 dt: 0.0015 error: 0.39671300024445344
RVE 615
Too many cells
RVE 616
itmax: 600 Cells: 94
it: 1 dt: 0.0015 error: 0.2845992973153667
RVE 617
itmax: 600 Cells: 775
it: 1 dt: 0.0015 error: 0.5009765070344072
RVE 618
itmax: 600 Cells: 510
it: 1 dt: 0.0015 error: 0.4202593664282221
RVE 619
itmax: 600 Cells: 177
it: 1 dt: 0.0015 error: 0.29431536663018387
RVE 620
itmax: 600 Cells: 443
it: 1 dt: 0.0015 error: 0.38168268569359515
RVE 621
itmax: 600 Cells: 220
it: 1 dt: 0.0015 error: 0.36434144707004135
RVE 622
Too many cells
RVE 623
itmax: 600 Cells: 129
it: 1 dt: 0.0015 error: 0.285

RVE 732
itmax: 600 Cells: 97
it: 1 dt: 0.0015 error: 0.27637735893259324
RVE 733
itmax: 200 Cells: 2866
it: 1 dt: 0.0015 error: 0.7306871034947565
RVE 734
itmax: 600 Cells: 446
it: 1 dt: 0.0015 error: 0.38505722829121225
RVE 735
itmax: 600 Cells: 331
it: 1 dt: 0.0015 error: 0.36480429897855615
RVE 736
itmax: 600 Cells: 226
it: 1 dt: 0.0015 error: 0.30519874514365325
RVE 737
itmax: 600 Cells: 131
it: 1 dt: 0.0015 error: 0.27597002288376443
RVE 738
itmax: 600 Cells: 115
it: 1 dt: 0.0015 error: 0.2775957808231701
RVE 739
itmax: 600 Cells: 633
it: 1 dt: 0.0015 error: 0.4243881425912541
RVE 740
itmax: 275 Cells: 1282
it: 1 dt: 0.0015 error: 0.5388421713416233
RVE 741
itmax: 200 Cells: 2545
it: 1 dt: 0.0015 error: 0.7021878500068551
RVE 742
itmax: 600 Cells: 724
it: 1 dt: 0.0015 error: 0.543726194070389
RVE 743
itmax: 600 Cells: 91
it: 1 dt: 0.0015 error: 0.23406261666894895
RVE 744
itmax: 600 Cells: 106
it: 1 dt: 0.0015 error: 0.24394696047256942
RVE 745
itmax: 600 Cells: 184
it: 1 dt: 0.00